In [41]:
import papermill as pm
import pandas as pd
import datetime
from shared.read_regulation import RegulationRetrievalHelper


In [42]:
import warnings
warnings.filterwarnings("ignore", 
                        category=DeprecationWarning, 
                        message=".*DataFrameGroupBy.apply operated on the grouping columns.*")

In [43]:
from config.config_loader import Config

config = Config()

In [44]:
alias = 'PR2'

In [ ]:
path_coefs_exc = config.get_path("data_path.elec.regulation.coefs_excesos")
path_ie = config.get_path("data_path.elec.regulation.ie")
path_ver_template = config.get_resolved_path("data_path.elec.verification_project.templates.verif")
path_estim_template = config.get_resolved_path("data_path.elec.verification_project.templates.estim")
compodem_a2_path = config.get_resolved_path("data_path.elec.verification_project.compodem_a2")

path_peajes_cargos_energia = config.get_resolved_path("data_path.elec.regulation.peajes_cargos_energia")
path_peajes_energia = config.get_resolved_path("data_path.elec.regulation.peajes_energia")
path_peajes_potencia = config.get_resolved_path("data_path.elec.regulation.peajes_potencia")
path_cargos_energia = config.get_resolved_path("data_path.elec.regulation.cargos_energia")
path_cargos_potencia = config.get_resolved_path("data_path.elec.regulation.cargos_potencia")
path_estructura_cargos = config.get_resolved_path("data_path.elec.regulation.estructura_cargos")
path_dto_peajes_electrointensivos = config.get_resolved_path("data_path.elec.regulation.dto_peajes_electrointensivos")

path_dto_peajes_electrointensivos


'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\regulation\\elec\\dto_peajes_electrointensivos.xlsx'

In [47]:
path_alias_elec = config.get_resolved_path("data_path.elec.customers.alias_elec")

In [ ]:
# Notebooks
# nb_elec_prices = config.get_resolved_path("notebooks.shared.elec_markets_2a")
# nb_elec_perd = config.get_resolved_path("notebooks.shared.perdidas")
# nb_elec_ssaa = config.get_resolved_path("notebooks.shared.ssaa")
# nb_load = config.get_resolved_path("notebooks.load_compilation.export")

# nb_market_matrix = config.get_resolved_path("notebooks.verification_project.build_matrix")
# nb_market_matrix_estim = config.get_resolved_path("notebooks.verification_project.build_matrix_estim")
# nb_contract_market_matrix = config.get_resolved_path("notebooks.verification_project.merge_market_contract")
# nb_matrix_estim = config.get_resolved_path("notebooks.verification_project.build_matrix_estim")
# nb_read_contracts = config.get_resolved_path("notebooks.verification_project.read_contract")
# nb_estim = config.get_resolved_path("notebooks.verification_project.estimacion")
# nb_estim_excel = config.get_resolved_path("notebooks.verification_project.estimacion_excel")
# nb_invoices = config.get_resolved_path("notebooks.verification_project.hunt_invoices")
# nb_verif = config.get_resolved_path("notebooks.verification_project.ver_nb")
# nb_contract_market_matrix

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\notebooks\\verification_project\\4B_Merge_Contract_OMIE.ipynb'

In [ ]:
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")

path_load_parquet = config.get_resolved_path("data_path.elec.processed.load_parquet")
path_load_elec_folder = config.get_resolved_path("data_path.elec.load_compilation.folder")
db_elec_path

'c:\\Users\\mikel.perez\\0_Python_projects_v2\\data\\processed\\elec\\facturas_elec.db'

In [50]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-07-04 14:29:23,886 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-04 14:29:23,886 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-04 14:29:23,890 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:29:23,893 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-04 14:29:23,895 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:29:23,895 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-04 14:29:23,898 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-04 14:29:23,958 INFO sqlalchemy.engine.Engine ROLLBACK


In [51]:
df_alias_elec = pd.read_excel(path_alias_elec)

In [52]:
cliente = 'Taghleef'
year = 2025

df_alias_fil = df_alias_elec[df_alias_elec['cliente'] == cliente]
# df_alias_fil = df_alias_fil[df_alias_fil['alias'].isin(['WARNER BS', 'WARNER BQ', 'WARNER DW'])]
# df_alias_fil = df_alias_fil[~df_alias_fil['cups'].isin(['ES0022000009138571BZ'])]
df_alias_fil

,cliente,alias,cups,tarifa,electrointensivo,contrato
28,Taghleef,TAGHLEEF,ES0031101457946001KR,6.1 TD,True,1


In [53]:
df_load = pd.read_parquet(path_load_parquet)

In [54]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

class OptimizePotencia:
    
    def __init__(self,
                 df_load: pd.DataFrame,
                 df_kp: pd.DataFrame,
                 df_tep: pd.DataFrame,
                 df_cargos_potencia: pd.DataFrame,
                 df_peajes_potencia: pd.DataFrame,
                 potencia: list):
        
        self.df_kp = df_kp
        self.df_tep = df_tep
        self.df_cargos_potencia = df_cargos_potencia
        self.df_peajes_potencia = df_peajes_potencia
        self.potencia = potencia

        df = df_load.copy()
        # df['load'] = df['load'] * 4
        self.df_load = df

        self.days = len(df_load['datetime'].dt.date.unique())
        self._merge_load_coefs()

    def _merge_load_coefs(self):
        # Use self.df_load and self.df_kp
        df_merged = pd.merge(self.df_load, self.df_kp, on='periodo', how='left')
        df_merged = pd.merge(df_merged, self.df_tep, on='periodo', how='left')
        self.df_op = df_merged
        return self

    def excess_calc(self, group):
        # Calculate the squared excess only if load exceeds potencia
        group['input'] = group.apply(
            lambda row: (row['load'] - row['potencia']) ** 2 if (row['load'] - row['potencia'] > 0) else 0, 
            axis=1
        )
        # Sum the squared differences per period and calculate the weighted excess
        list_pw = group.groupby('periodo')['input'].sum()
        tep = group.groupby('periodo')['tep_power_exc'].mean()
        kp = group.groupby('periodo')['kp_power_exc'].mean()
        excesos_potencia = (np.sqrt(list_pw) * kp * tep).sum()
        return excesos_potencia

    def objective_function(self, potencia):
        # Create a DataFrame for power, assuming period indices from 1 to 6
        df_power = pd.DataFrame(potencia, index=[period for period in range(1, 7)])
        df_power.reset_index(inplace=True)
        df_power.columns = ['periodo', 'potencia']

        df = pd.merge(self.df_op, df_power, on='periodo', how='left')

        # Use self.df_cargos_potencia and self.df_peajes_potencia
        coste_potencia = ((self.df_cargos_potencia['cargos_potencia'] + self.df_peajes_potencia['peajes_potencia'])
                          * self.days * df_power['potencia']).sum()
        
        df["month"] = df["datetime"].dt.to_period("M")
        monthly_results = df.groupby("month").apply(self.excess_calc).reset_index(name="monthly_result")
        # print(monthly_results['monthly_result'])
        
        # Sum only the 'monthly_result' column to get a scalar value
        total_cost_excess = monthly_results['monthly_result'].sum()

        self.coste_termino_fijo = coste_potencia
        self.coste_excesos = total_cost_excess
        
        return total_cost_excess + coste_potencia
    
    def minimize_potencia(self):

        n = len(self.potencia)

        # Bounds: power values must be non-negative
        bounds = [(0, None)] * n

        # Constraint: each value must be less than or equal to the next (x[i+1] - x[i] ≥ 0)
        constraints = [
            {'type': 'ineq', 'fun': lambda x, i=i: x[i+1] - x[i]} for i in range(n - 1)
        ]

        result = minimize(
            self.objective_function,
            self.potencia,
            method="SLSQP",
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000, 'disp': True}
        )

        print(result)
        return result


In [56]:
import pandas as pd
import os
from utils.utils_var import set_datetime_h
import datetime

os.listdir()

df = pd.read_excel('Cch_20240101_20241231 fábrica Málaga.xlsx')
df = df.head(-1)
df['hora'] = df['hora'].apply(lambda string: int(string[1:]))
df['load'] = df['load'] * 1000

df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')


df = set_datetime_h(date='date', hour='hora', load='load', df=df)

df['resolution'] = 'hourly'
df.columns = ['datetime', 'load', 'resolution']

path_spot_elec = config.get_path("data_path.elec.processed.spot_formatted")

df_spot_elec = pd.read_pickle(path_spot_elec)

df_spot_elec['dup'] = df_spot_elec.groupby('datetime').cumcount()
df['dup'] = df.groupby('datetime').cumcount()

df = pd.merge(df, df_spot_elec[['datetime', 'periodo', 'dup']], on=['datetime', 'dup'], how='left')

df.drop(columns=['dup'], inplace=True)

In [57]:
# Single optimization

df_load = df.copy()

# print(df_load)
# df_load = df_load[(df_load['resolution'] == 'quarter-hourly')
#                 #   (df_load['cups'] == cups)
#                   ]

# if df_load.empty:

#     df_load = df_load[(df_load['resolution'] == 'hourly')
#                   ]

df_load.sort_values('datetime', ascending=True, inplace=True)

date_fil = df_load['datetime'].max()
print(date_fil)

month_start = date_fil.month
year_start = date_fil.year - 1
date_start = datetime.datetime(date_fil.year - 1, month_start, 1,0,0)
print(date_start)

date_end = datetime.datetime(date_fil.year, month_start, 1,0,0) - datetime.timedelta(days=1)
date_end = datetime.datetime.combine(date_end.date(),datetime.time(23,55))
print(date_end)

df_load_fil = df_load[
                (df_load['datetime'] >= date_start) &
                (df_load['datetime'] <= date_end)
                ]
len(df_load_fil['datetime'].dt.date.unique())

2024-12-31 23:00:00
2023-12-01 00:00:00
2024-11-30 23:55:00


335

In [58]:
potencia = [17200, 17200, 17200, 17200, 17200, 17400]

tarifa = '6.2 TD'

In [59]:
Regulation = RegulationRetrievalHelper(
    tarifa=tarifa,
    path_peajes_cargos = path_peajes_cargos_energia,
    path_peajes_energia = path_peajes_energia,
    path_cargos_energia = path_cargos_energia,
    path_peajes_potencia = path_peajes_potencia,
    path_cargos_potencia = path_cargos_potencia,
    path_estructura_cargos = path_estructura_cargos,
    path_ie = path_ie,
    path_dto_peajes_electrointensivos = path_dto_peajes_electrointensivos,
    path_coefs_exc = path_coefs_exc
)

df_kp = Regulation._read_kp_power_exc_param(year=year)
df_tep = Regulation._read_tep_power_exc_param(year=year)

df_kp_2 = Regulation._read_kp_power_exc_param(year='2025 (2)')
df_tep_2 = Regulation._read_tep_power_exc_param(year='2025 (2)')

df_cargos_potencia = Regulation._read_peajes_slash_cargos(parameter='cargos_potencia',
                                    year=year)
df_peajes_potencia = Regulation._read_peajes_slash_cargos(parameter='peajes_potencia',
                                    year=year)



In [ ]:
OpNewReg = OptimizePotencia(
    df_load=df_load_fil,
    df_kp=df_kp_2,
    df_tep=df_tep_2,
    df_cargos_potencia=df_cargos_potencia,
    df_peajes_potencia=df_peajes_potencia,
    potencia=potencia
)

optimized_power_new = OpNewReg.minimize_potencia()
POWEROPNEW = list(optimized_power_new.x)
old_cost = OpNewReg.objective_function(potencia)
old_power_cost = OpNewReg.coste_termino_fijo
old_cost_excess = OpNewReg.coste_excesos
optimized_cost = OpNewReg.objective_function(POWEROPNEW)
new_power_cost = OpNewReg.coste_termino_fijo
new_cost_excess = OpNewReg.coste_excesos
dif = optimized_cost - old_cost

items = [cliente, alias] + [P for P in POWEROPNEW] + [P for P in potencia]+ [new_power_cost, new_cost_excess, old_power_cost, old_cost_excess, dif]
optimized_list = []
for item in items:
    optimized_list.append(item)
print(len(items))
df_optimizator = pd.DataFrame(columns=\
    ['cliente', 'alias'] + [f'potencia_p{i} op' for i in range(1,7)] + [f'potencia_p{i} old' for i in range(1,7)] + ['new_power_cost', 'new_cost_excess', 'old_power_cost', 'old_cost_excess', 'dif'])
df_optimizator = pd.DataFrame([optimized_list], columns=df_optimizator.columns)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 584362.9697808626
            Iterations: 44
            Function evaluations: 322
            Gradient evaluations: 44
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 584362.9697808626
       x: [ 1.603e+04  1.644e+04  1.644e+04  1.644e+04  1.644e+04
            1.644e+04]
     nit: 44
     jac: [ 0.000e+00 -5.062e+00  2.359e+00  1.328e+00  1.000e+00
            3.672e-01]
    nfev: 322
    njev: 44
19


In [63]:
df_optimizator = pd.DataFrame(columns=\
    ['cliente', 'alias'] + [f'potencia_p{i} op' for i in range(1,7)] + [f'potencia_p{i} old' for i in range(1,7)] + ['new_power_cost', 'new_cost_excess', 'old_power_cost', 'old_cost_excess', 'dif'])
df_optimizator = pd.DataFrame([optimized_list], columns=df_optimizator.columns)
df_optimizator.T

,0
cliente,Taghleef
alias,PR2
potencia_p1 op,16030.41656
potencia_p2 op,16437.071585
potencia_p3 op,16437.071585
potencia_p4 op,16437.071585
potencia_p5 op,16437.071585
potencia_p6 op,16437.071585
potencia_p1 old,17200
potencia_p2 old,17200


In [62]:
df_optimizator

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,Taghleef,PR2,16030.41656,16437.071585,16437.071585,16437.071585,16437.071585,16437.071585,17200,17200,17200,17200,17200,17400,572719.889341,11643.08044,607070.511392,0.0,-22707.541611
